# Phi-4 Mini ONNX समानांतर फ़ंक्शन कॉलिंग ट्यूटोरियल

यह नोटबुक दिखाती है कि Phi-4 Mini को ONNX Runtime GenAI के साथ समानांतर फ़ंक्शन कॉलिंग के लिए कैसे उपयोग करें। फ़ंक्शन कॉलिंग मॉडल को उपयोगकर्ता अनुरोधों के आधार पर बाहरी टूल और API को बुद्धिमानी से बुलाने की अनुमति देती है।

## अवलोकन

इस ट्यूटोरियल में, आप सीखेंगे:
- Phi-4 Mini को ONNX Runtime GenAI के साथ सेट करना
- फ्लाइट और होटल बुकिंग के लिए फ़ंक्शन स्कीमा परिभाषित करना
- संरचित आउटपुट के लिए Lark व्याकरण के साथ निर्देशित जनरेशन का उपयोग करना
- जटिल यात्रा बुकिंग परिदृश्यों के लिए समानांतर फ़ंक्शन कॉल निष्पादित करना

## आवश्यकताएँ

इस नोटबुक को चलाने से पहले, सुनिश्चित करें कि आपने:
- Phi-4 Mini ONNX मॉडल डाउनलोड कर लिया है
- `onnxruntime-genai` पैकेज इंस्टॉल कर लिया है
- फ़ंक्शन कॉलिंग अवधारणाओं की बुनियादी समझ


## चरण 1: आवश्यक लाइब्रेरी आयात करें

सबसे पहले, हम अपनी फ़ंक्शन कॉलिंग कार्यान्वयन के लिए आवश्यक लाइब्रेरी आयात करेंगे।


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## चरण 2: मॉडल सेटअप और कॉन्फ़िगरेशन

अब हम Phi-4 Mini ONNX मॉडल को कॉन्फ़िगर करेंगे। सुनिश्चित करें कि आप पथ को अपने वास्तविक मॉडल डायरेक्टरी से बदलें।


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## चरण 3: जेनरेशन पैरामीटर कॉन्फ़िगर करें

मॉडल के आउटपुट व्यवहार को नियंत्रित करने के लिए जेनरेशन पैरामीटर सेट करें। ये सेटिंग्स फंक्शन कॉलिंग के लिए निश्चित और केंद्रित प्रतिक्रियाएं सुनिश्चित करती हैं।


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## चरण 4: उपलब्ध फ़ंक्शनों को परिभाषित करें

यहां हम उन फ़ंक्शनों को परिभाषित करते हैं जिन्हें हमारा AI सहायक कॉल कर सकता है। इस उदाहरण में, हमारे पास यात्रा से संबंधित दो फ़ंक्शन हैं:
1. **booking_flight_tickets**: हवाई अड्डों के बीच फ्लाइट बुक करने के लिए
2. **booking_hotels**: होटल में ठहरने की व्यवस्था बुक करने के लिए

फ़ंक्शन की परिभाषाएं OpenAI के फ़ंक्शन कॉलिंग स्कीमा प्रारूप का पालन करती हैं।


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## चरण 5: व्याकरण निर्माण सहायक फ़ंक्शन

ये सहायक फ़ंक्शन हमारे फ़ंक्शन परिभाषाओं को Lark व्याकरण प्रारूप में परिवर्तित करते हैं, जिसका उपयोग ONNX Runtime GenAI द्वारा निर्देशित निर्माण के लिए किया जाता है। यह सुनिश्चित करता है कि मॉडल सही JSON प्रारूप में मान्य फ़ंक्शन कॉल्स उत्पन्न करे।


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## चरण 6: व्याकरण निर्माण का परीक्षण करें

आइए अपनी व्याकरण निर्माण फ़ंक्शन्स का परीक्षण करें ताकि यह देखा जा सके कि वे हमारे टूल परिभाषाओं को सही प्रारूप में कैसे बदलते हैं।


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## चरण 7: सिस्टम प्रॉम्प्ट और जनरेटर तैयार करें

अब हम सिस्टम प्रॉम्प्ट बनाएंगे जो मॉडल को उपलब्ध टूल्स के बारे में बताएगा और निर्देशित जनरेशन पैरामीटर के साथ जनरेटर सेट करेंगे।


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## चरण 8: गाइडेड जनरेशन के साथ जनरेटर को प्रारंभ करें

अब हम अपने कॉन्फ़िगर किए गए पैरामीटर के साथ जनरेटर बनाएंगे और गाइडेड जनरेशन के लिए Lark व्याकरण लागू करेंगे।


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## चरण 9: समानांतर फ़ंक्शन कॉलिंग का परीक्षण करें

अब चलिए एक जटिल यात्रा बुकिंग परिदृश्य के साथ हमारी सेटअप का परीक्षण करते हैं, जिसमें कई फ़ंक्शनों को कॉल करना आवश्यक है।


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### फ़ंक्शन कॉल्स उत्पन्न करें

मॉडल अब हमारे उपयोगकर्ता अनुरोध के आधार पर संरचित फ़ंक्शन कॉल्स उत्पन्न करेगा। निर्देशित जनरेशन की मदद से, आउटपुट मान्य JSON प्रारूप में होगा जिसे सीधे निष्पादित किया जा सकता है।

**अपेक्षित आउटपुट**: मॉडल को निम्नलिखित के लिए फ़ंक्शन कॉल्स उत्पन्न करने चाहिए:
1. `booking_flight_tickets` जिसमें बीजिंग (PEK) से पेरिस (CDG) की जानकारी हो
2. `booking_hotels` जिसमें पेरिस में आवास की जानकारी हो

नीचे दिए गए सेल को चलाएं और लाइव जनरेशन देखें:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## निष्कर्ष

🎉 **बधाई हो!** आपने ONNX Runtime GenAI का उपयोग करके Phi-4 Mini के साथ समानांतर फ़ंक्शन कॉलिंग को सफलतापूर्वक लागू कर लिया है।

### आपने क्या सीखा:

1. **मॉडल सेटअप**: ONNX Runtime GenAI के साथ Phi-4 Mini को कैसे कॉन्फ़िगर करें
2. **फ़ंक्शन परिभाषा**: AI फ़ंक्शन कॉलिंग के लिए फ़ंक्शन स्कीमा कैसे परिभाषित करें
3. **निर्देशित जनरेशन**: संरचित आउटपुट जनरेशन के लिए Lark व्याकरण का उपयोग कैसे करें
4. **समानांतर फ़ंक्शन कॉल्स**: जटिल अनुरोधों को संभालने के लिए कई फ़ंक्शन कॉल्स कैसे करें

### मुख्य लाभ:

- ✅ **संरचित आउटपुट**: निर्देशित जनरेशन मान्य JSON फ़ंक्शन कॉल्स सुनिश्चित करता है
- ✅ **समानांतर प्रोसेसिंग**: एक ही अनुरोध में कई फ़ंक्शन कॉल्स को संभालें
- ✅ **उच्च प्रदर्शन**: ONNX Runtime अनुकूलित इनफेरेंस प्रदान करता है
- ✅ **लचीला स्कीमा**: फ़ंक्शन परिभाषाओं को जोड़ना या संशोधित करना आसान है

### संसाधन:

- [Phi-4 Mini दस्तावेज़](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI दस्तावेज़](https://onnxruntime.ai/docs/genai/)
- [फ़ंक्शन कॉलिंग सर्वोत्तम प्रथाएँ](https://platform.openai.com/docs/guides/function-calling)



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
